In [ ]:
import pyttsx3
import numpy as np
import cv2
weights = 'yolov3.weights'
txt = 'yolov3.txt'
config = 'yolov3.cfg'
import os
engine = pyttsx3.init()
engine.setProperty('rate', 150)

def video_to_image(video):
    l = os.listdir('Frames/')
    for i in l:
        i='Frames/'+i
        os.remove(i)
        
    sec = 0
    vidcap = cv2.VideoCapture(video)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            cv2.imwrite("Frames/"+"frame "+str(sec)+" sec.jpg", image)     # save frame as JPG file
        return hasFrames

    frameRate = 1 
    success = getFrame(sec)
    while success:
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)
        

import sys
filename = ''
from tkinter import *
root = Tk()
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
import tkinter.messagebox as tkMessageBox
import tkinter.simpledialog as tkSimpleDialog
from tkinter.simpledialog import Dialog

def browsefunc():
    global filename
    filename = filedialog.askopenfilename()
    print(filename)
    pathlabel.config(text=filename)
    
browsebutton = Button(root, text="Browse", command=browsefunc)
browsebutton.pack()

pathlabel = Label(root)
pathlabel.pack()
root.mainloop()
video_to_image(filename)


def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h): #for rectangle creation

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    


lst = os.listdir('Frames/')  #acess the frames and scale down
for i in lst:
    i = 'Frames/' + i
    image = cv2.imread(i)

    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392
    lst = []
    classes = None

    with open(txt, 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    COLORS = np.random.uniform(0, 255, size=(len(classes), 3)) #random color picking

    net = cv2.dnn.readNet(weights, config) #deep nn 

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False) #pre process and sending it to deep nn

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4 #non max supression


    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5: #verifying the conf and then find the center of the rectangle to be read
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        lst.append(classes[class_ids[i]]) #calling the prediction fun

    
    cv2.imshow("object detection", image)
    cv2 .waitKey(0)
    
#cv2.imwrite("object-detection.jpg", image)
    cv2.destroyAllWindows()
    for i in lst:
        print(i)
        engine.say(i)
        engine.runAndWait()